<a href="https://colab.research.google.com/github/Dalia-Sher/Generating-Facial-Expressions-Wasserstein-Conditional-BiGAN/blob/Shir/biGAN_with_new_disc_and_gen_10000_epochs_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers import MaxPooling2D, concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K

import matplotlib.pyplot as plt
import pandas as pd

import numpy as np

In [65]:
data = pd.read_csv('fer2013.csv')
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [66]:
data.Usage.value_counts()

Training       28709
PublicTest      3589
PrivateTest     3589
Name: Usage, dtype: int64

In [67]:
num_classes = 7
img_width = 48
img_height = 48

In [68]:
data.shape

(35887, 3)

In [69]:
X = data['pixels']
y = data['emotion']

In [70]:
X_train = []
for i in X:
    X_train.append([int(j) for j in i.split()])

X_train = np.array(X_train)/255.0

In [71]:
X_train = X_train.reshape(X_train.shape[0], img_width, img_height, 1)
X_train = X_train.astype('float32')

In [72]:
X_train.shape

(35887, 48, 48, 1)

In [73]:
img_rows, img_cols, channels, latent_dim = 48, 48 , 1 , 100

In [74]:
img_shape = (img_rows, img_cols, channels)


In [75]:
optimizer = Adam(0.0002, 0.5)


In [88]:
def build_encoder(latent_dim, img_shape):
        model = Sequential()

        model.add(Flatten(input_shape=img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(latent_dim))

        model.summary()

        img = Input(shape=img_shape)
        z = model(img)

        return Model(img, z)

In [89]:

def build_generator(latent_dim, img_shape):
  model = Sequential()
  n_nodes = 128 * 12 * 12
  model.add(Dense(512, input_dim=latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))
  model.add(Dense(np.prod(img_shape), activation='tanh'))
  model.add(Reshape(img_shape))

  model.summary()

  z = Input(shape=(latent_dim,))
  gen_img = model(z)

  return Model(z, gen_img)

In [90]:
def build_discriminator(latent_dim, img_shape):

  z = Input(shape=(latent_dim, ))
  img = Input(shape=img_shape)
  d_in = concatenate([z, Flatten()(img)])

  model = Dense(1024)(d_in)
  model = LeakyReLU(alpha=0.2)(model)
  model = Dropout(0.5)(model)
  model = Dense(1024)(model)
  model = LeakyReLU(alpha=0.2)(model)
  model = Dropout(0.5)(model)
  model = Dense(1024)(model)
  model = LeakyReLU(alpha=0.2)(model)
  model = Dropout(0.5)(model)
  validity = Dense(1, activation="sigmoid")(model)

  return Model([z, img], validity)


In [94]:
# Build and compile the discriminator
discriminator = build_discriminator(latent_dim,img_shape)
discriminator.compile(loss=['binary_crossentropy'],
    optimizer=optimizer,
    metrics=['accuracy'])

# Build the generator
generator = build_generator(latent_dim, img_shape)

# Build the encoder
encoder = build_encoder(latent_dim, img_shape)

# The part of the bigan that trains the discriminator and encoder
discriminator.trainable = False

# Generate image from sampled noise
z = Input(shape=(latent_dim, ))
img_ = generator(z)

# Encode image
img = Input(shape=img_shape)
z_ = encoder(img)

# Latent -> img is fake, and img -> latent is valid
fake = discriminator([z, img_])
valid = discriminator([z_, img])

# Set up and compile the combined model
# Trains generator to fool the discriminator
bigan_generator = Model([z, img], [fake, valid])
bigan_generator.compile(loss=['binary_crossentropy', 'binary_crossentropy'],
    optimizer=optimizer)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_75 (Dense)             (None, 512)               51712     
_________________________________________________________________
leaky_re_lu_53 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
batch_normalization_26 (Batc (None, 512)               2048      
_________________________________________________________________
dense_76 (Dense)             (None, 512)               262656    
_________________________________________________________________
leaky_re_lu_54 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
batch_normalization_27 (Batc (None, 512)               2048      
_________________________________________________________________
dense_77 (Dense)             (None, 2304)            

In [107]:
def train( X_train, bigan_generator, generator, discriminator, encoder,latent_dim, epochs, batch_size=128, intervals=100):

  # Adversarial ground truths
  valid = np.ones((batch_size, 1))
  fake = np.zeros((batch_size, 1))

  for epoch in range(epochs):
    # ---------------------
    #  Train Discriminator
    # ---------------------

    # Sample noise and generate img
    z = np.random.normal(size=(batch_size, latent_dim))
    imgs_ = generator.predict(z)

    # Select a random batch of images and encode
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    imgs = X_train[idx]
    z_ = encoder.predict(imgs)

    # Train the discriminator (img -> z is valid, z -> img is fake)
    d_loss_real = discriminator.train_on_batch([z_, imgs], valid)
    d_loss_fake = discriminator.train_on_batch([z, imgs_], fake)
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    # ---------------------
    #  Train Generator
    # ---------------------

    # Train the generator (z -> img is valid and img -> z is is invalid)
    g_loss = bigan_generator.train_on_batch([z, imgs], [valid, fake])

    # Plot the progress
    print ("%d [D loss: %f, acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss[0]))

    # If at save interval => save generated image samples
    if epoch % intervals == 0:
        sample_interval(latent_dim, epoch)

In [111]:
def sample_interval(latent_dim, epoch):
  r, c = 5, 5
  z = np.random.normal(size=(25, latent_dim))
  gen_imgs = generator.predict(z)

  gen_imgs = 0.5 * gen_imgs + 0.5

  fig, axs = plt.subplots(r, c)
  cnt = 0
  for i in range(r):
      for j in range(c):
          axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
          axs[i,j].axis('off')
          cnt += 1
  fig.savefig(r"C:\Users\shir2\Desktop\Shir\MSc\Deep_generative_models\project\biGan100321\expression_facial_%d.png" % epoch)
  plt.close()

In [112]:

train(X_train, bigan_generator, generator, discriminator, encoder,latent_dim, epochs=10000, batch_size=32, intervals=500)

Streaming output truncated to the last 5000 lines.
5000 [D loss: 0.699262, acc: 42.19%] [G loss: 1.402981]
5001 [D loss: 0.690119, acc: 56.25%] [G loss: 1.449844]
5002 [D loss: 0.679202, acc: 57.81%] [G loss: 1.446833]
5003 [D loss: 0.709680, acc: 39.06%] [G loss: 1.426565]
5004 [D loss: 0.720747, acc: 37.50%] [G loss: 1.443969]
5005 [D loss: 0.700244, acc: 51.56%] [G loss: 1.420806]
5006 [D loss: 0.682973, acc: 51.56%] [G loss: 1.447749]
5007 [D loss: 0.689298, acc: 50.00%] [G loss: 1.475334]
5008 [D loss: 0.703433, acc: 43.75%] [G loss: 1.511910]
5009 [D loss: 0.681264, acc: 50.00%] [G loss: 1.496504]
5010 [D loss: 0.712668, acc: 42.19%] [G loss: 1.385087]
5011 [D loss: 0.705247, acc: 42.19%] [G loss: 1.412539]
5012 [D loss: 0.689638, acc: 48.44%] [G loss: 1.443337]
5013 [D loss: 0.709148, acc: 48.44%] [G loss: 1.460539]
5014 [D loss: 0.697874, acc: 54.69%] [G loss: 1.432274]
5015 [D loss: 0.724686, acc: 34.38%] [G loss: 1.413199]
5016 [D loss: 0.686455, acc: 45.31%] [G loss: 1.41019